<a href="https://colab.research.google.com/github/rezPirayesh/Customer-prediction/blob/main/Customer_Prediction_PyTorch.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
print("Hello, Colab works!")
import torch
print(torch.__version__)
print("GPU available:", torch.cuda.is_available())

Hello, Colab works!
2.6.0+cu124
GPU available: True


In [ ]:
import pandas as pd
df = pd.read_csv('data.csv', encoding='ISO-8859-1')
print(df.head())  # First 5 rows
print(df.info())  # Columns, types
print(df['CustomerID'].notnull().value_counts())  # Purchase indicator

  InvoiceNo StockCode                          Description  Quantity  \
0    536365    85123A   WHITE HANGING HEART T-LIGHT HOLDER         6   
1    536365     71053                  WHITE METAL LANTERN         6   
2    536365    84406B       CREAM CUPID HEARTS COAT HANGER         8   
3    536365    84029G  KNITTED UNION FLAG HOT WATER BOTTLE         6   
4    536365    84029E       RED WOOLLY HOTTIE WHITE HEART.         6   

      InvoiceDate  UnitPrice  CustomerID         Country  
0  12/1/2010 8:26       2.55     17850.0  United Kingdom  
1  12/1/2010 8:26       3.39     17850.0  United Kingdom  
2  12/1/2010 8:26       2.75     17850.0  United Kingdom  
3  12/1/2010 8:26       3.39     17850.0  United Kingdom  
4  12/1/2010 8:26       3.39     17850.0  United Kingdom  
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100247 entries, 0 to 100246
Data columns (total 8 columns):
 #   Column       Non-Null Count   Dtype  
---  ------       --------------   -----  
 0   InvoiceNo   

In [ ]:
import torch
import torch.nn as nn
model = nn.Sequential(
    nn.Linear(2, 4),
    nn.ReLU(),
    nn.Linear(4, 1),
    nn.Sigmoid()
)
print("Model created:", model)

Model created: Sequential(
  (0): Linear(in_features=2, out_features=4, bias=True)
  (1): ReLU()
  (2): Linear(in_features=4, out_features=1, bias=True)
  (3): Sigmoid()
)


In [2]:
import pandas as pd
df = pd.read_csv('data.csv', encoding='ISO-8859-1')
df = df.dropna(subset=['CustomerID'])
df = df[df['Quantity'] > 0]
df['InvoiceDate'] = pd.to_datetime(df['InvoiceDate'])
print("Cleaned dataset:")
print(df.info())
print(df.head())

Cleaned dataset:
<class 'pandas.core.frame.DataFrame'>
Index: 54345 entries, 0 to 87647
Data columns (total 8 columns):
 #   Column       Non-Null Count  Dtype         
---  ------       --------------  -----         
 0   InvoiceNo    54345 non-null  object        
 1   StockCode    54345 non-null  object        
 2   Description  54345 non-null  object        
 3   Quantity     54345 non-null  int64         
 4   InvoiceDate  54345 non-null  datetime64[ns]
 5   UnitPrice    54345 non-null  float64       
 6   CustomerID   54345 non-null  float64       
 7   Country      54345 non-null  object        
dtypes: datetime64[ns](1), float64(2), int64(1), object(4)
memory usage: 3.7+ MB
None
  InvoiceNo StockCode                          Description  Quantity  \
0    536365    85123A   WHITE HANGING HEART T-LIGHT HOLDER         6   
1    536365     71053                  WHITE METAL LANTERN         6   
2    536365    84406B       CREAM CUPID HEARTS COAT HANGER         8   
3    536365    8

In [3]:
df['TotalPrice'] = df['Quantity'] * df['UnitPrice']
customer_data = df.groupby('CustomerID').agg({
    'TotalPrice': 'sum',
    'InvoiceNo': 'nunique',
    'Quantity': 'sum'
}).reset_index()
customer_data.columns = ['CustomerID', 'TotalSpent', 'PurchaseFrequency', 'TotalItems']
customer_data['Purchased'] = 1
print("Customer features:")
print(customer_data.head())
print(customer_data.info())

Customer features:
   CustomerID  TotalSpent  PurchaseFrequency  TotalItems  Purchased
0     12346.0    77183.60                  1       74215          1
1     12347.0     1187.18                  2         634          1
2     12348.0     1120.24                  2        1855          1
3     12350.0      334.40                  1         197          1
4     12356.0     2271.62                  1        1216          1
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1450 entries, 0 to 1449
Data columns (total 5 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   CustomerID         1450 non-null   float64
 1   TotalSpent         1450 non-null   float64
 2   PurchaseFrequency  1450 non-null   int64  
 3   TotalItems         1450 non-null   int64  
 4   Purchased          1450 non-null   int64  
dtypes: float64(2), int64(3)
memory usage: 56.8 KB
None
